<a href="https://colab.research.google.com/github/Ahmed-Ragab/machine-learning-zoomcamp/blob/main/lead_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [4]:
!wget $data -O Lead_Score.csv

--2025-10-15 22:02:43--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘Lead_Score.csv’

Lead_Score.csv      100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-15 22:02:43 (6.90 MB/s) - ‘Lead_Score.csv’ saved [80876/80876]



In [5]:
df = pd.read_csv('Lead_Score.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


**Data preparation**

In [6]:
# Check if the missing values are presented in the features.
df.isnull().sum()

,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [7]:
# Replace null values in categorical features with 'NA'
categorical_features = df.select_dtypes(include='object').columns
df[categorical_features] = df[categorical_features].fillna('NA')

In [8]:
# Replace null values in numerical features with 0.0
numerical_features = list(df.select_dtypes(include='number').columns)
if 'converted' in numerical_features:
    numerical_features.remove('converted')
df[numerical_features] = df[numerical_features].fillna(0.0)

**Question 1**

In [9]:
# What is the most frequent observation (mode) for the column industry?
df['industry'].groupby(df['industry']).count().sort_values(ascending=False)



,industry
industry,
retail,203
finance,200
other,198
education,187
healthcare,187
technology,179
manufacturing,174
NA,134


**Question 2**

In [10]:
# Create the correlation matrix for the numerical features of your dataset
corr_matrix = df[numerical_features].corr()
corr_matrix



,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [11]:
df[numerical_features].corrwith(df.converted).abs()


,0
number_of_courses_viewed,0.435914
annual_income,0.053131
interaction_count,0.374573
lead_score,0.193673


**Split the data**

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Split your data in train/val/test sets with 60%/20%/20% distribution. set the seed to 42
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [14]:
# reset index for each df
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [15]:
# Build y data (target variable) for each df "train, val, and test"
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [16]:
# Delete the target variable from the DFs
del df_train['converted']
del df_val['converted']
del df_test['converted']

**Question 3**

In [17]:
from sklearn.metrics import mutual_info_score

In [18]:
# Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_train_full.converted)

df_mi = df_train_full[categorical_features].apply(mutual_info_converted_score)
df_mi = round(df_mi.sort_values(ascending=False),2)
df_mi


,0
lead_source,0.03
employment_status,0.01
industry,0.01
location,0.00


**Question 4**

In [19]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [20]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

# Use Scikit-Learn to encode categorical features
dv = DictVectorizer(sparse=False)

# Combine categorical and numerical features
all_features = list(categorical_features) + list(numerical_features)

train_dict = df_train[all_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[all_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [21]:
# Train a logistic regression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [22]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
round(model.score(X_val, y_val),2)

0.7

**Question 5. Feature selection**

In [23]:
# find the least useful feature using the feature elimination technique
# Calculate the accuracy on the validation dataset.
model.coef_[0].round(3) # This is the w (weights)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [24]:
# Model intercept
model.intercept_[0].round(3)

np.float64(-0.069)

In [25]:
# get features name
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [26]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'annual_income': np.float64(-0.0),
 'employment_status=NA': np.float64(-0.015),
 'employment_status=employed': np.float64(0.034),
 'employment_status=self_employed': np.float64(0.003),
 'employment_status=student': np.float64(0.012),
 'employment_status=unemployed': np.float64(-0.103),
 'industry=NA': np.float64(-0.025),
 'industry=education': np.float64(0.049),
 'industry=finance': np.float64(-0.02),
 'industry=healthcare': np.float64(-0.013),
 'industry=manufacturing': np.float64(-0.003),
 'industry=other': np.float64(-0.009),
 'industry=retail': np.float64(-0.032),
 'industry=technology': np.float64(-0.016),
 'interaction_count': np.float64(0.311),
 'lead_score': np.float64(0.051),
 'lead_source=NA': np.float64(0.02),
 'lead_source=events': np.float64(-0.012),
 'lead_source=organic_search': np.float64(-0.012),
 'lead_source=paid_ads': np.float64(-0.115),
 'lead_source=referral': np.float64(0.08),
 'lead_source=social_media': np.float64(-0.03),
 'location=NA': np.float64(0.004),
 'l

In [67]:
small = ['industry', 'employment_status']

In [68]:
dict_train_small = df_train[small].to_dict(orient='records')
dict_val_small = df_val[small].to_dict(orient='records')

In [69]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dict_train_small)


DictVectorizer(sparse=False)

In [70]:
dv_small.get_feature_names_out()

array(['employment_status=NA', 'employment_status=employed',
       'employment_status=self_employed', 'employment_status=student',
       'employment_status=unemployed', 'industry=NA',
       'industry=education', 'industry=finance', 'industry=healthcare',
       'industry=manufacturing', 'industry=other', 'industry=retail',
       'industry=technology'], dtype=object)

In [71]:
X_train_small = dv_small.transform(dict_train_small)

In [72]:
# train model
model_small = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_small.fit(X_train_small, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [73]:
# get intercept
w0 = model_small.intercept_[0]
w0

np.float64(0.3615861811993119)

In [74]:
# get coeffient
w = model_small.coef_[0]
w

array([-0.15957961,  0.41000573,  0.24902483,  0.34078925, -0.47865401,
       -0.26638478,  0.91118749, -0.19166274,  0.04391234,  0.09943592,
        0.04502039, -0.18353636, -0.09638609])

In [75]:
dict(zip(dv_small.get_feature_names_out(), w))

{'employment_status=NA': np.float64(-0.15957960973585522),
 'employment_status=employed': np.float64(0.41000572953731906),
 'employment_status=self_employed': np.float64(0.24902482622191655),
 'employment_status=student': np.float64(0.3407892466165004),
 'employment_status=unemployed': np.float64(-0.478654011440644),
 'industry=NA': np.float64(-0.2663847778750124),
 'industry=education': np.float64(0.9111874919604901),
 'industry=finance': np.float64(-0.1916627382387818),
 'industry=healthcare': np.float64(0.04391234214315025),
 'industry=manufacturing': np.float64(0.09943592482148035),
 'industry=other': np.float64(0.045020387494980704),
 'industry=retail': np.float64(-0.18353635993375877),
 'industry=technology': np.float64(-0.09638608917330822)}

In [77]:
X_val_small = dv_small.transform(dict_val_small)
model_small.score(X_val_small, y_val)

# Original Accuracy: 0.6348122866894198
# Accuracy wihout industry: 0.6143344709897611
# Accuracy wihout employment_status: 0.6518771331058021
# Accuracy wihout lead_score: 0.6245733788395904

0.6245733788395904

In [79]:
Original_Accuracy = 0.6348122866894198
Accuracy_without_industry = 0.6143344709897611
Accuracy_without_employment_status = 0.6518771331058021
Accuracy_without_lead_score = 0.6245733788395904

In [82]:
Original_Accuracy - Accuracy_without_industry

0.020477815699658675

In [83]:
Original_Accuracy - Accuracy_without_employment_status


-0.017064846416382284

In [84]:
Original_Accuracy - Accuracy_without_lead_score

0.010238907849829393

**Question 6. Parameter tuning**

In [86]:
# train a regularized logistic regression
all_features = list(categorical_features) + list(numerical_features)

full_train_dict = df_train_full[all_features].to_dict(orient='records')
#X_train = dv.fit_transform(train_dict)

#val_dict = df_val[all_features].to_dict(orient='records')
#X_val = dv.transform(val_dict)


In [87]:
dv = DictVectorizer(sparse=False)


In [88]:
X_full_train = dv.fit_transform(train_dict)


In [89]:
y_full_train = df_train_full.converted.values

In [91]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_full_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [93]:
# Calculate accuracy with each feature removed
accuracy_scores = {}
for feature_to_remove in all_features:
    features_subset = [f for f in all_features if f != feature_to_remove]

    train_dict_subset = df_train[features_subset].to_dict(orient='records')
    dv_subset = DictVectorizer(sparse=False)
    X_train_subset = dv_subset.fit_transform(train_dict_subset)

    val_dict_subset = df_val[features_subset].to_dict(orient='records')
    X_val_subset = dv_subset.transform(val_dict_subset)

    model_subset = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_subset.fit(X_train_subset, y_train)

    accuracy_subset = model_subset.score(X_val_subset, y_val)
    accuracy_scores[feature_to_remove] = accuracy_subset

accuracy_diff = {feature: accuracy - acc_subset for feature, acc_subset in accuracy_scores.items()}

In [94]:
# Identify the feature with the smallest difference
least_useful_feature = min(accuracy_diff, key=accuracy_diff.get)
min_accuracy_diff = accuracy_diff[least_useful_feature]

print(f"Original Accuracy: {accuracy:.4f}")
print("\nAccuracy after removing each feature:")
for feature, acc_subset in accuracy_scores.items():
    print(f"- {feature}: {acc_subset:.4f} (Difference: {accuracy - acc_subset:.4f})")

print(f"\nFeature with the smallest difference: {least_useful_feature}")
print(f"Smallest accuracy difference: {min_accuracy_diff:.4f}")

Original Accuracy: 0.6997

Accuracy after removing each feature:
- lead_source: 0.7031 (Difference: -0.0034)
- industry: 0.6997 (Difference: 0.0000)
- employment_status: 0.6962 (Difference: 0.0034)
- location: 0.7099 (Difference: -0.0102)
- number_of_courses_viewed: 0.5563 (Difference: 0.1433)
- annual_income: 0.8532 (Difference: -0.1536)
- interaction_count: 0.5563 (Difference: 0.1433)
- lead_score: 0.7065 (Difference: -0.0068)

Feature with the smallest difference: annual_income
Smallest accuracy difference: -0.1536


In [92]:
# Evaluate the model on the validation set
accuracy = model.score(X_val, y_val)
print(f"Accuracy on validation set: {accuracy:.4f}")

Accuracy on validation set: 0.6997
